In [120]:
from pathlib import Path
import pandas as pd
import numpy as np

In [121]:
LOAD_DIR = Path(Path.cwd(), "data")

In [122]:
tmax_df = pd.read_fwf(Path(LOAD_DIR, 'tmax.txt'))

In [123]:
with open (Path(LOAD_DIR, 'tmax.txt')) as f:
    data = (f.readlines()[5:])

In [124]:
lst_lsts = []
for i in range(len(data)):
    _temp = data[i].split(" ")
    lst_lsts.append([i for i in _temp if i])

In [125]:
df = pd.DataFrame(lst_lsts)

In [127]:
def clean_df(raw_df:pd.DataFrame) -> pd.DataFrame:
    raw_df.columns = raw_df.iloc[0]
    df = raw_df[1:]
    df = df.replace(to_replace = ["---","\n", None], value = np.nan)
    df['ann'] = df['ann\n'].str.replace("\n", "")
    df = df.drop('ann\n', axis = 1, errors = 'ignore')
    df = df.apply(pd.to_numeric)
    return df

df = clean_df(df)

In [129]:
df.head()

,year,jan,feb,mar,apr,may,jun,jul,aug,sep,oct,nov,dec,win,spr,sum,aut,ann
1,1884,7.3,6.8,8.5,10.1,14.5,17.1,18.8,20.2,17.0,11.8,7.5,5.8,NaN,11.02,18.73,12.10,12.14
2,1885,4.3,7.3,7.2,10.8,11.8,17.0,19.4,16.8,15.0,9.3,7.5,5.9,5.75,9.91,17.76,10.58,11.04
3,1886,3.7,3.4,6.1,10.5,13.1,16.4,18.7,18.5,16.0,12.9,8.6,4.1,4.36,9.88,17.87,12.51,11.04
4,1887,4.8,6.9,6.7,9.9,13.0,19.8,20.7,19.1,14.5,10.0,6.6,4.9,5.23,9.88,19.87,10.37,11.44
5,1888,5.5,3.7,5.2,9.0,14.1,16.3,16.0,16.9,15.4,11.4,8.9,7.1,4.70,9.41,16.28,11.91,10.52
